# Coffee Usage

Python notebook for analysing the coffee usage in the NBI HEP group, and determining the length of the Christmas vacation from a fit to it. The exercise lies in writing a stepwise fit function, and making it work on real data.

For a period in 2009-2010, the usage of the old coffey machine in the NBI HEP group was (somewhat irregularly) monitored. Below is the count of total number of cups of coffey ever brewed at given dates. We will mainly focus on the data from around Christmas...


- 28479   4/11-2009     NOTE: This day, we in the following define as day 0! 
- 28674  13/11-2009 
- 28777  18/11-2009 
- 28964  25/11-2009 
- 29041  27/11-2009 
- 29374  10/12-2009 
- ~29650   8/ 1-2010
- 30001  29/ 1-2010 (?)
- 30221   8/ 2-2010
- 30498  21/ 2-2010
- 32412  17/ 5-2010
- 33676  11/ 8-2010
- 34008   9/ 9-2010`

### Author: 
- Troels C. Petersen (Niels Bohr Institute)

### Date:    
- 29-11-2021 (latest update)

***

In [ ]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats

Load the external functions:

In [ ]:
sys.path.append('../../../External_Functions')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure

and define the parameters of the program:

In [ ]:
save_plots = False

## Fitting functions:

First we define a simple linear function:

In [ ]:
def func_coffee_simple(x, p0, p1): 
    return p0 + p1*x

Now YOU should add an extra parameter, which is the length of the vacation:

In [ ]:
def func_coffee_advanced(x, p0, p1, p2):
    
    # Define when the vacation is (i.e. which points are before and which are after).
    # This could in itself be a fitting parameter, but for now we keep things simple.
    start_vacation = 40.0       # I.e. 40 days after the first date, which 4th of November
    
    # Here goes your function in terms of p0, p1, and p2 (vacation length)...
    return p0 + p1*x + p2*x*x/100.0       # Just a silly function!
    
# Numpy-way of "vectorizing" the function above:
func_coffee_advanced_vec = np.vectorize(func_coffee_advanced)

The reason why we are using the last line is to allow `func_coffee_advanced` to take Numpy arrays as inputs, so we do not have to loop over entire lists.

## Coffee Usage

Data set (So small that we will not use a seperate file for it!). We define 4th of November 2009 to be day 0, and count from there.

In [ ]:
days = np.array([     0,     9,    14,    21,    23,    36,    65,    76,    86,    99,   194,   280,   309 ])
cups = np.array([ 28479, 28674, 28777, 28964, 29041, 29374, 29650, 30001, 30221, 30498, 32412, 33676, 34008 ])
edays = np.zeros_like(days)
ecups = np.ones_like(cups)*30.0   # I estimate the uncertainty to be 30 cups, but perhaps you disagree?
                                  # Certainly, it is NOT the square root, as it is not at all a Poisson number!

# Print the number of days:
for day, cup in zip(days, cups):
    print(f"days: {day:3.0f}    cups: {cup:5.0f}")

## Fit and plot graph on screen:

In [ ]:
# Define the relevant range:
xmin, xmax = 60, 105
mask_range = (xmin < days) & (days < xmax)

# Fit the data points in the range (xmin, xmax)
chi2_object_coffee = Chi2Regression(func_coffee_simple, days[mask_range], cups[mask_range], ecups[mask_range])
minuit_coffee = Minuit(chi2_object_coffee, p0=28000.0, p1=20.0)
minuit_coffee.migrad();
p0, p1 = minuit_coffee.values[:]

print("Simple fit")
for name in minuit_coffee.parameters:
    print("Fit value: {0} = {1:.5f} +/- {2:.5f}".format(name, minuit_coffee.values[name], minuit_coffee.errors[name]))

Plot the data along with the fit on top of the data:

In [ ]:
# Plot the data:
fig, ax = plt.subplots(figsize=(12, 8))
ax.errorbar(days, cups, xerr=edays, yerr=ecups, fmt='o', label='Data')
ax.set(xlim=(-20, 400), xlabel='Days', ylabel='Cups', title='Coffe usage as function of time')

# Plot the fit on top:
x_fit = np.linspace(xmin, xmax, 1000)
y_fit = func_coffee_simple(x_fit, *minuit_coffee.args)
ax.plot(x_fit, y_fit, '-', label='Simple Fit')

# Get the ChiSquare probability:
chi2_lin = minuit_coffee.fval
ndof_lin = len(days[mask_range]) - len(minuit_coffee.values[:])
chi2_prob_lin = stats.chi2.sf(chi2_lin, ndof_lin)

# Include fit results in the plot:
d = {'Chi2': chi2_lin,
     'Ndof': ndof_lin,
     'Prob': chi2_prob_lin,
     'p0': [minuit_coffee.values['p0'], minuit_coffee.errors['p0']],
     'p1': [minuit_coffee.values['p1'], minuit_coffee.errors['p1']],
    }

text = nice_string_output(d, extra_spacing=2, decimals=3)
add_text_to_ax(0.02, 0.97, text, ax, fontsize=14);

## Advanced fit:
Try to fit the advanced function in the range (-10, 110):

In [ ]:
xmin, xmax = -10, 110
mask_range = (xmin < days) & (days < xmax)

# fit the data points in the range (xmin, xmax)
chi2_object_coffee2 = Chi2Regression(func_coffee_advanced, days[mask_range], cups[mask_range], ecups[mask_range])
minuit_coffee2 = Minuit(chi2_object_coffee2, p0=28500, p1=25, p2=14.0)
minuit_coffee2.migrad();

p0, p1, p2 = minuit_coffee2.values[:]
print("Advanced fit")
for name in minuit_coffee2.parameters:
    print("Fit value: {0} = {1:.5f} +/- {2:.5f}".format(name, minuit_coffee2.values[name], minuit_coffee2.errors[name]))

and plot the advanced fit:

In [ ]:
x_fit = np.linspace(xmin, xmax, 1000)
y_fit = func_coffee_advanced_vec(x_fit, *minuit_coffee2.values[:])
ax.plot(x_fit, y_fit, '-', label='Advanced Fit')
fig

In [ ]:
print(f"\n\nThe fitted effective vacation length was: {minuit_coffee2.values['p2']:4.1f} +- {minuit_coffee2.errors['p2']:3.1f} days!\n")

This is of course absurd, but that is because p2 does not represent the length of the Christmas vacation. Change the advanced fit, such that it does, and see if you get a reasonable vacation length out.

# Questions:

First, only consider the first 100 days of data.

0. From the ChiSquare of the linear fit to four points (65-100 days), is the estimated uncertainty of 30 cups reasonable?

1. Assuming the error of 30 cups, do the numbers follow the hypothesis of constant use? Quantify this, and find out how large the error has to be (roughly), for this hypothesis to be credible.

2. Does taking into account Christmas vacation looks like it will improve the above hypothesis? Can you actually fit the length of vacation? Try to rewrite the function "func_coffee_advanced", such that it includes "if", dividing the function into two linear functions with the same slope, that has an interval with no usage (i.e. the Christmas vacation). What length do you obtain? And how much better is the model?

Now consider the whole data set.

3. The total number of cups of coffey ever brewed was 36716, after which the old coffey machine was decommissioned. From the above data, estimate when this happened (including error!). And when would you estimate that the coffey machine was commissioned originally?